In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
diabetes_data = pd.read_csv('C:\IDA\DataCleaningProject\data/diabetes_data.csv', sep = ',')
diabetes_clean = diabetes_data.copy()
diabetes_clean = diabetes_clean.drop_duplicates()
low_information_cols = []
for col in diabetes_clean.columns:
    top_freq=diabetes_clean[col].value_counts(normalize=True).max()
    nununique_ratio=diabetes_clean[col].nunique()/diabetes_clean[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
    if nununique_ratio > 0.95:
        low_information_cols.append(col)
diabetes_clean=diabetes_clean.drop(axis=1, columns=low_information_cols)
null_cols=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
diabetes_clean['Glucose']=diabetes_clean['Glucose'].replace({'0':np.nan, 0:np.nan})
diabetes_clean['BloodPressure']=diabetes_clean['BloodPressure'].replace({'0':np.nan, 0:np.nan})
diabetes_clean['SkinThickness']=diabetes_clean['SkinThickness'].replace({'0':np.nan, 0:np.nan})
diabetes_clean['Insulin']=diabetes_clean['Insulin'].replace({'0':np.nan, 0:np.nan})
diabetes_clean['BMI']=diabetes_clean['BMI'].replace({'0':np.nan, 0:np.nan})
diabetes_clean=diabetes_clean.dropna(axis=1, thresh=diabetes_clean.shape[0]*0.7)
diabetes_clean=diabetes_clean.dropna(thresh=diabetes_clean.shape[1]-2)
values={
     'Pregnancies':diabetes_clean['Pregnancies'].median(), 
     'Glucose':diabetes_clean['Glucose'].median(),
     'BloodPressure':diabetes_clean['BloodPressure'].median(),
     'SkinThickness':diabetes_clean['SkinThickness'].median(),
     'BMI':diabetes_clean['BMI'].median(),
     'DiabetesPedigreeFunction':diabetes_clean['DiabetesPedigreeFunction'].median(),
     'Age':diabetes_clean['Age'].median(),
     'Outcome':diabetes_clean['Outcome'].median()
     }
diabetes_clean=diabetes_clean.fillna(values)
def outliers_iqr(data,feature):
    x=data[feature]
    quantile_1, quantile_3=x.quantile(0.25), x.quantile(0.75)
    iqr=quantile_3-quantile_1
    lower_bound=quantile_1-(iqr*1.5)
    upper_bound=quantile_3+(iqr*1.5)
    outliers=data[(x<lower_bound)|(x>upper_bound)]
    cleaned=data[(x>lower_bound)&(x<upper_bound)]
    return outliers, cleaned
outliers_iqr, cleaned_iqr = outliers_iqr(diabetes_clean, 'DiabetesPedigreeFunction')
def outliers_z_score(data,feature,log_scale=False):
    if log_scale:
        x=np.log(data[feature])
    else:
        x=data[feature]
    mu=x.mean()
    sigma=x.std()
    lower_bound=mu-3*sigma
    upper_bound=mu+3*sigma
    outliers=data[(x<lower_bound)|(x>upper_bound)]
    cleaned=data[(x>lower_bound)&(x<upper_bound)]
    return outliers, cleaned
outliers_z, cleaned_z = outliers_z_score(diabetes_clean, 'DiabetesPedigreeFunction', log_scale=True)
display (outliers_iqr.shape[0] - outliers_z.shape[0])

29